# Scraping Canadian Immigration Processing Times

This notebook scrapes the immigration times processing page - to get the processing time for applicants applying for PR under the Economic Migration class

(using Playwright)

In [1]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import datetime
import json
import csv

In [2]:
playwright = await async_playwright().start()

In [3]:
browser = await playwright.chromium.launch(headless = False)

In [4]:
page = await browser.new_page()

In [5]:
await page.goto("https://www.canada.ca/en/immigration-refugees-citizenship/services/application/check-processing-times.html")

<Response url='https://www.canada.ca/en/immigration-refugees-citizenship/services/application/check-processing-times.html' request=<Request url='https://www.canada.ca/en/immigration-refugees-citizenship/services/application/check-processing-times.html' method='GET'>>

In [6]:
try: 
    await page.click("xpath=/html/body/aside/div/div/ul/li[2]/button", timeout=1000)

except: 
    pass


In [7]:
await page.select_option("select#wb-auto-23", index=2)

['Economic immigration\n              ']

## Get Economic Migration categories + processing times

In [8]:
for index in range(2,12):

    if index == 4:
        pass

    elif index == 5:
        pass
    
    else:

        data = {}
        
        today = datetime.date.today()

        await page.select_option("select#wb-auto-48", index=index)

        type = await page.select_option("select#wb-auto-48", index=index)

        type = type[0]

        type = type.replace(" ", "_")

        await page.click("xpath=/html/body/main/div[1]/div[3]/div[4]/div[1]/div/form/button")

        await page.wait_for_timeout(3000)

        html = await page.content()

        doc = BeautifulSoup(html, 'html.parser')

        time = doc.find_all('span', {'class': 'wb-init'})
        
        data["time"] =  time[0].text 
        data["date_recorded"] =  time[1].text 

        print(data)
        
        with open(f'./data/{type}.csv', 'a', newline='') as csvfile:
            fieldnames = ['time', 'date_recorded']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(data)

 

{'time': '4 months', 'date_recorded': 'April 18, 2023'}
{'time': '5 months', 'date_recorded': 'April 18, 2023'}
{'time': '44 months', 'date_recorded': 'April 18, 2023'}
{'time': '67 months', 'date_recorded': 'April 18, 2023'}
{'time': '27 months', 'date_recorded': 'April 18, 2023'}
{'time': 'Not enough data', 'date_recorded': 'April 18, 2023'}
{'time': '20 months', 'date_recorded': 'April 18, 2023'}
{'time': '35 months', 'date_recorded': 'April 18, 2023'}


In [9]:
await browser.close()